In [1]:
import pandas as pd
import numpy as np

Note this notebook is to clean Findex data for use in Tableau - used to make a spatial map of financial inclusion across sub saharan africa

In [2]:
df = pd.read_excel('03-data/Global Findex Database.xlsx',
                  usecols = ["Unnamed: 0", "Unnamed: 1", "Unnamed: 2", "Unnamed: 3", "Unnamed: 4", 
                   "Financial institution account (% age 15+) "])

In [4]:
df_ssa = df[(df['Unnamed: 3'] == "Sub-Saharan Africa (excluding high income)") & (df["Unnamed: 0"] == 2017)]

In [5]:
df_ssa.rename(columns = {"Unnamed: 0" : "year",
                         "Unnamed: 1" : "country_label", 
                         "Unnamed: 2" : "country",
                         "Unnamed: 3" : "region",
                         "Unnamed: 4" : "income_level"},inplace = True)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [6]:
df_ssa.loc[(df_ssa["country"] == 'Congo, Dem. Rep.'), ["country"]] = "Democratic Republic of the Congo"
df_ssa.loc[(df_ssa["country"] == 'Congo, Rep.'), ["country"]] = "Republic of the Congo"

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [7]:
import pycountry
len(pycountry.countries)

list(pycountry.countries)[0:1]

[Country(alpha_2='AW', alpha_3='ABW', name='Aruba', numeric='533')]

In [8]:
ssa_list = ["Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cape Verde", 
            "Cameroon", "Central African Republic", "Chad", "Comoros", "Democratic Republic of the Congo", 
            "Republic of the Congo", "Cote d'Ivoire", "Djibouti", "Equatorial Guinea", "Eritrea", "Ethiopia", 
            "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Madagascar", 
            "Malawi", "Mali", "Mauritania", "Mauritius", "Mozambique", "Namibia", "Niger", "Nigeria", "Rwanda", 
            "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", "South Sudan", 
            "Sudan", "Swaziland", "Tanzania", "Togo", "Uganda", "Zambia", "Zimbabwe"]

In [9]:
input_countries = ['Benin', 'Burkina Faso', 'Botswana', 'Central African Republic',
       "Cote d'Ivoire", 'Cameroon', 'Democratic Republic of the Congo',
       'Republic of the Congo', 'Ethiopia', 'Gabon', 'Ghana', 'Guinea',
       'Kenya', 'Liberia', 'Lesotho', 'Madagascar', 'Mali', 'Mozambique',
       'Mauritania', 'Mauritius', 'Malawi', 'Namibia', 'Niger', 'Nigeria',
       'Rwanda', 'Senegal', 'Sierra Leone', 'South Sudan', 'Chad', 'Togo',
       'Tanzania', 'Uganda', 'South Africa', 'Zambia', 'Zimbabwe']

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

df_ssa['alpha2'] = [countries.get(country, 'Unknown code') for country in input_countries]

<ipython-input-9-0a53f9154b01>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ssa['alpha2'] = [countries.get(country, 'Unknown code') for country in input_countries]


In [10]:
df_ssa.loc[(df_ssa["country"] == "Cote d'Ivoire"), ["alpha2"]] = "CI"
df_ssa.loc[(df_ssa["country"] == "Democratic Republic of the Congo"), ["alpha2"]] = "CD"
df_ssa.loc[(df_ssa["country"] == "Republic of the Congo"), ["alpha2"]] = "CG"
df_ssa.loc[(df_ssa["country"] == "Tanzania"), ["alpha2"]] = "TZ"

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [123]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='tk')

alphas = ['BJ', 'BF', 'BW', 'CF', 'CI', 'CM', 'CD', 'CG', 'ET', 'GA', 'GH',
       'GN', 'KE', 'LR', 'LS', 'MG', 'ML', 'MZ', 'MR', 'MU', 'MW', 'NA',
       'NE', 'NG', 'RW', 'SN', 'SL', 'SS', 'TD', 'TG', 'TZ', 'UG', 'ZA',
       'ZM', 'ZW']

def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        df_ssa['geolocate'] = [(loc.latitude, loc.longitude) for country in alphas]
    except:
        # Return missing value
        return np.nan

In [126]:
df_ssa.to_csv('ssa_findex.csv')